<a target="_blank" href="https://colab.research.google.com/github/UpstageAI/cookbook/blob/main/cookbooks/upstage/Solar-Full-Stack LLM-101/05_3_OracleDB.ipynb">
<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Retrieval Augmented Generation (RAG) Baseline
## Overview  
In this time, we will check the baseline code.
The goal of this project is to provide students with hands-on experience in handling and enhancing Large Language Models (LLMs) provided by [**Upstage**](https://www.upstage.ai) (Solar).

You can use any engineering method for improving benchmark performance excluding direct training (Fine-tuning).

*Collecting data directly related to the test set is considered cheating e.g., using MMLU-pro dataset or EWHA.pdf for KB*

In [1]:
!pip install openai

In [2]:
# pip install openai

from openai import OpenAI # openai==1.52.2

client = OpenAI(
    api_key="up_EoF0I0CzeHxuDYmf0we544GMPCFIT",
    base_url="https://api.upstage.ai/v1"
)

stream = client.chat.completions.create(
    model="solar-pro2",
    messages=[
        {
            "role": "user",
            "content": "Hi, how are you?"
        }
    ],
    stream=True,
)

for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")

# Use with stream=False
# print(stream.choices[0].message.content)

Hello! I'm just a large language model, so I don't have feelings, but I'm here and ready to help you. How can I assist you today? Is there something you'd like to talk about or ask?  

(If you're asking about my operational status, I'm functioning normally—feel free to ask me any questions!)

In [3]:
response = client.chat.completions.create(
    model="solar-pro2",
    messages=[
        {
            "role": "user",
            "content": "Hi, how are you?"
        }
    ],
    stream=False,
)
print(response.choices[0].message.content)

I'm just a large language model, so I don't have feelings, but I'm here and ready to help you! How can I assist you today? Is there something specific you'd like to talk about or ask about?  

(And nice to meet you! 😊)  

If you're curious, you can ask me questions, request explanations, or even have a casual conversation. Let me know!


# Baseline

In [4]:
!pip3 install -qU python-dotenv PyPDF2 langchain langchain-community langchain-core langchain-text-splitters langchain_upstage oracledb python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 87.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
transformers 4.57.1 requires tokenizers<=0.23.0,>=0.22.0, but you have tokenizers 0.20.3 which is in

In [ ]:
# Additional Contents : https://wikidocs.net/253106

In [5]:
# set parameters
from google.colab import drive
drive.mount('/content/drive')

api_key = "up_EoF0I0CzeHxuDYmf0we544GMPCFIT"
data_path = "/content/drive/MyDrive/NLP" # folder path containing ewah.pdf and samples.csv

Mounted at /content/drive


#ewha.pdf 로드


In [6]:
'''
from langchain_upstage import UpstageDocumentParseLoader
import os

UPSTAGE_API_KEY = api_key

layzer = UpstageDocumentParseLoader(api_key=UPSTAGE_API_KEY,file_path=os.path.join(data_path, 'ewha.pdf'), ocr="layout")

docs = layzer.load()  # or layzer.lazy_load()
print("총 페이지 수:", len(docs))
print(docs[0].page_content[:500])
'''

'\nfrom langchain_upstage import UpstageDocumentParseLoader\nimport os\n\nUPSTAGE_API_KEY = api_key\n\nlayzer = UpstageDocumentParseLoader(api_key=UPSTAGE_API_KEY,file_path=os.path.join(data_path, \'ewha.pdf\'), ocr="layout")\n\ndocs = layzer.load()  # or layzer.lazy_load()\nprint("총 페이지 수:", len(docs))\nprint(docs[0].page_content[:500])\n'

위의 upstagedocumentparseloader는 pdf를 하나로 인식해서 기존 랭체인의 pdf로더 임포트

In [75]:
from langchain_community.document_loaders import PyPDFLoader
import os

loader = PyPDFLoader(os.path.join(data_path, "ewha.pdf"))
docs = loader.load()
docs = [d for d in docs if int(d.metadata["page"]) < 38]
print("총 페이지 수:", len(docs))
print(docs[0].page_content[:500])


총 페이지 수: 38
2 - 2 - 1이화여자대학교 학칙
1946. 8. 15.  제정
2017. 8. 16.  개정
제1장 총칙
제1조(목적) 본교는 대한민국의 교육이념과 기독교정신을 바탕으로 하여 학술의 깊은 이론과 
그 광범하고 정밀한 응용방법을 교수․연구하며 , 인격을 도야하여 국가와 인류사회의 발전에 
공헌할 수 있는 지도여성을 양성함을 목적으로 한다.
제2조(명칭) 본교는 이화여자대학교라 부른다 .
제3조(위치) 본교는 서울특별시 서대문구 이화여대길 52에 둔다. (개정 2013.2.25.)
제2장 편제
제4조(대학 및 대학원 ) ① 본교에는 다음 각 호의 대학을 둔다.  
1. 인문과학대학 , 사회과학대학 , 자연과학대학 , 엘텍공과대학 , 음악대학 , 조형예술대학 , 사범
대학, 경영대학 , 신산업융합대학 , 의과대학 , 간호대학 , 약학대학 , 스크랜튼대학 (이하 “각 
대학”이라 한다)  (개정 2016.6.16.)  
2. 호크마 (HOKMA) 교양대학
② 본교에는 대학원 , 국제대학


#ewha.pdf 청킹 시작

본문/부칙 나누기(부칙에 표가 많아서)

In [60]:
#전체 텍스트 병합
full_text = "\n".join([d.page_content for d in docs])

# "부칙" 등장 지점 찾기
split_index = full_text.find("부칙")
print("부칙 시작 인덱스:", split_index)

#본문 / 부칙 분리
if split_index != -1:
    main_text = full_text[:split_index]
    appendix_text = full_text[split_index:]
else:
    main_text = full_text
    appendix_text = ""

print("본문 길이:", len(main_text))
print("부칙 길이:", len(appendix_text))

부칙 시작 인덱스: 20919
본문 길이: 20919
부칙 길이: 20872


본문 먼저 청킹

In [118]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document

main_doc = [Document(page_content=main_text)]
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1200,
    chunk_overlap=300,
    separators=["별표", "\n\n제", "\n제", "\n\n", "\n", ". ", " "],
)

main_chunks = text_splitter.split_documents(main_doc)
print("본문 청크 수:", len(main_chunks))
print(main_chunks[0].page_content[:300])

본문 청크 수: 23
2 - 2 - 1이화여자대학교 학칙
1946. 8. 15.  제정
2017. 8. 16.  개정
제1장 총칙
제1조(목적) 본교는 대한민국의 교육이념과 기독교정신을 바탕으로 하여 학술의 깊은 이론과 
그 광범하고 정밀한 응용방법을 교수․연구하며 , 인격을 도야하여 국가와 인류사회의 발전에 
공헌할 수 있는 지도여성을 양성함을 목적으로 한다.
제2조(명칭) 본교는 이화여자대학교라 부른다 .
제3조(위치) 본교는 서울특별시 서대문구 이화여대길 52에 둔다. (개정 2013.2.25.)
제2장 편제
제4조(대학 및 대학원 ) ① 본교에는


In [119]:
print(main_chunks[0])

page_content='2 - 2 - 1이화여자대학교 학칙
1946. 8. 15.  제정
2017. 8. 16.  개정
제1장 총칙
제1조(목적) 본교는 대한민국의 교육이념과 기독교정신을 바탕으로 하여 학술의 깊은 이론과 
그 광범하고 정밀한 응용방법을 교수․연구하며 , 인격을 도야하여 국가와 인류사회의 발전에 
공헌할 수 있는 지도여성을 양성함을 목적으로 한다.
제2조(명칭) 본교는 이화여자대학교라 부른다 .
제3조(위치) 본교는 서울특별시 서대문구 이화여대길 52에 둔다. (개정 2013.2.25.)
제2장 편제
제4조(대학 및 대학원 ) ① 본교에는 다음 각 호의 대학을 둔다.  
1. 인문과학대학 , 사회과학대학 , 자연과학대학 , 엘텍공과대학 , 음악대학 , 조형예술대학 , 사범
대학, 경영대학 , 신산업융합대학 , 의과대학 , 간호대학 , 약학대학 , 스크랜튼대학 (이하 “각 
대학”이라 한다)  (개정 2016.6.16.)  
2. 호크마 (HOKMA) 교양대학
② 본교에는 대학원 , 국제대학원 , 통역번역대학원 , 경영전문대학원 , 법학전문대학원 , 교육대
학원, 디자인대학원 ,  사회복지대학원 , 신학대학원 , 정책과학대학원 , 공연예술대학원 , 임상보
건융합대학원 , 임상치의학대학원 , 외국어교육특수대학원을 둔다(이하 “각 대학원 ”이라 한다).  
(개정 2016.6.16., 2017.5.15.)
[전문개정  2015.11.27.]
제5조(학부․학과․전공 및 정원) ① 각 대학, 학부, 학과, 전공 및 모집단위별 입학정원은'


부칙 청킹 파트

In [120]:
appendix_doc = [Document(page_content=appendix_text)]
with open("/content/drive/MyDrive/NLP/ewha_appendix.txt", "w") as f:
    f.write(appendix_text)
print("✅ 부칙 부분 따로 저장 완료")

✅ 부칙 부분 따로 저장 완료


In [122]:
appendix_doc = [Document(page_content=appendix_text)]
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1200,
    chunk_overlap=300,
    separators=["부칙", "별표", "\n\n제", "\n제", "\n\n", "\n", ". ", " "],
)
appendix_chunks = text_splitter.split_documents(appendix_doc)
print("부칙 청크 수:", len(appendix_chunks))
print(appendix_chunks[0].page_content[:300])

부칙 청크 수: 23
부칙(1946. 8. 15  제정)
본 학칙은 1946 년 3월 1일부터 시행한다 .
  부칙
① 본 학칙은 1961 년 3월 1일부터 시행한다 .
② 본 학칙 시행에 관한 세칙은 총장이 정한다 .
  부칙(문관행 1040.1-1549, 1963. 12. 16  개정) 
본 학칙은 1964 년 3월 1일부터 시행한다 .
  부칙(문고대 1041.3-16, 1965. 1. 5  개정) 
ⓛ 이 학칙은 1965 년 1월 10일부터 시행한다 .
② 이 학칙 시행 당시 문리과대학에 소속하는 가정학과와 사범대학에 소속하는 가정학과는 
폐과하


부칙 청킹1) 수위되는 학사 테이블 - 연도별로

In [123]:
import pandas as pd
import glob
import re

base_path = "/content/drive/MyDrive/NLP/"

file_paths = [
    base_path + "degree_2006.csv",
    base_path + "degree_2009.csv",
    base_path + "degree_2017.csv",
    base_path + "degree_latest.csv",
]

sentences = []

for file in file_paths:

    # 연도 추출
    match = re.search(r"(20\d{2})", file)
    if match:
        year = match.group(1) + " 개정"
    else:
        year = "최신 개정"



    df = pd.read_csv(file)   # 헤더 있는 버전 유지

    for idx, row in df.iterrows():

        # 대학 / 학사 / 전공 추출하면서 NaN 또는 float 자동 스킵
        college = row.get("대학")
        degree = row.get("학사")
        major = row.get("전공")

        # 하나라도 비어 있으면 스킵
        if pd.isna(college) or pd.isna(degree) or pd.isna(major):

            continue

        # 문자열로 변환 + strip()
        college = str(college).strip()
        degree = str(degree).strip()
        major = str(major).strip()

        # 문장 만들기
        sentence = f"{college}의 {major} 전공은 {degree} 학위를 수여한다. ({year})"
        sentences.append(sentence)

print("\n생성된 문장 수:", len(sentences))
print("\n샘플 10개:\n")
for s in sentences[:10]:
    print(s)

# 저장
pd.DataFrame({"sentence": sentences}).to_csv(
    base_path + "degree_sentences_all.csv",
    index=False, encoding="utf-8-sig"
)

print("\n저장 완료 → degree_sentences_all.csv")


생성된 문장 수: 160

샘플 10개:

사회과학대학의 사회복지학 전공은 문학사 학위를 수여한다. (2006 개정)
자연과학대학의 분자생명과학 전공은 이학사 학위를 수여한다. (2006 개정)
공과대학의 컴퓨터학 전공은 공학사 학위를 수여한다. (2006 개정)
공과대학의 정보통신학 전공은 공학사 학위를 수여한다. (2006 개정)
공과대학의 건축학(4년제) 전공은 공학사 학위를 수여한다. (2006 개정)
공과대학의 건축학(5년제) 전공은 건축학사 학위를 수여한다. (2006 개정)
공과대학의 환경학 전공은 공학사 학위를 수여한다. (2006 개정)
음악대학의 건반악기 전공은 음악학사 학위를 수여한다. (2006 개정)
음악대학의 관현악 전공은 음악학사 학위를 수여한다. (2006 개정)
음악대학의 성악 전공은 음악학사 학위를 수여한다. (2006 개정)

저장 완료 → degree_sentences_all.csv


In [124]:
from langchain.schema import Document

df = pd.read_csv("/content/drive/MyDrive/NLP/degree_sentences_all.csv")

degree_docs = [
    Document(page_content=row["sentence"])
    for _, row in df.iterrows()
]

print("문서 개수:", len(degree_docs))

문서 개수: 160


부칙 청킹2) 입학정원 테이블

In [125]:
df = pd.read_csv("/content/drive/MyDrive/NLP/capacity_2019.csv")
print(df.columns)

Index(['year', 'college', 'department', 'major', 'quota'], dtype='object')


In [126]:
capacity_files = {
    "2014": "/content/drive/MyDrive/NLP/capacity_2014.csv",
    "2015": "/content/drive/MyDrive/NLP/capacity_2015.csv",
    "2016": "/content/drive/MyDrive/NLP/capacity_2016.csv",
    "2017": "/content/drive/MyDrive/NLP/capacity_2017.csv",
    "2018": "/content/drive/MyDrive/NLP/capacity_2018.csv",
    "2019": "/content/drive/MyDrive/NLP/capacity_2019.csv",
}

sentences = []

for year, path in capacity_files.items():
    print(f"📄 Processing: {path}")

    df = pd.read_csv(path)

    for idx, row in df.iterrows():
        college = str(row["college"]).strip()
        department = str(row["department"]).strip()
        major = str(row["major"]).strip()
        quota = row["quota"]

        # Null, nan 제외
        if quota == "Null" or pd.isna(quota):
            continue

        # ★ 1) 전공이 있는 경우
        if major != "nan" and major.strip() != "":
            text = f"{year}학년도 기준, {college} {department}의 {major} 입학정원은 {quota}명이다."

        # ★ 2) 전공은 없고 department만 있는 경우 (학부 정원)
        elif department != "nan" and department.strip() != "":
            text = f"{year}학년도 기준, {college} {department}의 입학정원은 {quota}명이다."

        # ★ 3) department도 없는 경우 (단과대 전체 정원)
        else:
            text = f"{year}학년도 기준, {college} 전체 입학정원은 {quota}명이다."

        sentences.append(text)

print("생성된 문장 수:", len(sentences))

print("\n샘플 10개:\n")
for s in sentences[:10]:
    print(s)

# 저장
pd.DataFrame({"sentence": sentences}).to_csv(
    "/content/drive/MyDrive/NLP/capacity_sentences_all.csv",
    index=False, encoding="utf-8-sig"
)

print("\n📁 저장됨 → capacity_sentences_all.csv")

📄 Processing: /content/drive/MyDrive/NLP/capacity_2014.csv
📄 Processing: /content/drive/MyDrive/NLP/capacity_2015.csv
📄 Processing: /content/drive/MyDrive/NLP/capacity_2016.csv
📄 Processing: /content/drive/MyDrive/NLP/capacity_2017.csv
📄 Processing: /content/drive/MyDrive/NLP/capacity_2018.csv
📄 Processing: /content/drive/MyDrive/NLP/capacity_2019.csv
생성된 문장 수: 272

샘플 10개:

2014학년도 기준, 인문과학대학 인문과학부의 입학정원은 433명이다.
2014학년도 기준, 인문과학대학 기독교학부의 기독교학 전공 입학정원은 20명이다.
2014학년도 기준, 인문과학대학 인문과학대학 전체의 입학정원은 453명이다.
2014학년도 기준, 사회과학대학 사회과학부의 입학정원은 337명이다.
2014학년도 기준, 사회과학대학 언론·홍보·영상학부의 입학정원은 90명이다.
2014학년도 기준, 사회과학대학 사회과학대학 전체의 입학정원은 427명이다.
2014학년도 기준, 자연과학대학 수리물리과학부의 입학정원은 124명이다.
2014학년도 기준, 자연과학대학 화학생명분자과학부의 입학정원은 171명이다.
2014학년도 기준, 자연과학대학 자연과학대학 전체의 입학정원은 295명이다.
2014학년도 기준, 공과대학 컴퓨터공학과의 입학정원은 71명이다.

📁 저장됨 → capacity_sentences_all.csv


본문+degree_sentences_all + capacity_sentences_all 파일을 합쳐서 **최종 RAG KB(knowledge base)**를 만드는 전체 과정

In [127]:
degree_df = pd.read_csv("/content/drive/MyDrive/NLP/degree_sentences_all.csv")
degree_docs = [Document(page_content=s) for s in degree_df["sentence"].tolist()]
print("학위표 문장 수:", len(degree_docs))

# ---- 입학정원표 문장 로드 ----
capacity_df = pd.read_csv("/content/drive/MyDrive/NLP/capacity_sentences_all.csv")
capacity_docs = [Document(page_content=s) for s in capacity_df["sentence"].tolist()]
print("정원표 문장 수:", len(capacity_docs))


# ===========================================
# 3. 전체 KB 통합
# ===========================================
all_docs = main_chunks + degree_docs + capacity_docs+appendix_chunks
print("총 KB 문서 수:", len(all_docs))


학위표 문장 수: 160
정원표 문장 수: 272
총 KB 문서 수: 478


In [128]:
from langchain_upstage import UpstageEmbeddings

emb = UpstageEmbeddings(
    api_key="up_EoF0I0CzeHxuDYmf0we544GMPCFIT",
    model="solar-embedding-1-large"
)

In [129]:
!pip install faiss-cpu

In [130]:
from langchain.vectorstores import FAISS

vector_store = FAISS.from_documents(all_docs, emb)

vector_store.save_local("/content/drive/MyDrive/NLP/ewha_vectorstore")


In [131]:
vector_store = FAISS.load_local(
    "/content/drive/MyDrive/NLP/ewha_vectorstore",
    emb,
    allow_dangerous_deserialization=True
)

In [132]:
query = "2019학년도 소프트웨어학부 입학정원 몇 명이야?"
retrieved_docs = vector_store.similarity_search(query, k=5)

for d in retrieved_docs:
    print(d.page_content)

2019학년도 기준, 엘텍공과대학 소프트웨어학부의 입학정원은 105명이다.
2018학년도 기준, 엘텍공과대학 소프트웨어학부의 입학정원은 105명이다.
2017학년도 기준, 엘텍공과대학 소프트웨어학부의 입학정원은 105명이다.
2016학년도 기준, 공과대학 컴퓨터공학과의 입학정원은 71명이다.
2015학년도 기준, 공과대학 컴퓨터공학과의 입학정원은 71명이다.


In [133]:
# read samples.csv file

import pandas as pd

def read_data(data_path):
    data = pd.read_csv(data_path)
    prompts = data['prompts']
    answers = data['answers']
    # returns two lists: prompts and answers
    return prompts, answers

In [134]:
prompts, answers = read_data(os.path.join(data_path, 'testset.csv'))

In [135]:
from langchain_core.prompts import PromptTemplate
from langchain_upstage import ChatUpstage

llm = ChatUpstage(
    api_key="up_EoF0I0CzeHxuDYmf0we544GMPCFIT",
    model="solar-pro2",
)

prompt_template = PromptTemplate.from_template(
    """
    Please provide the most correct answer from the following context.
    If the answer is not present in the context, please write "The information is not present in the context."
    ---
    Question: {question}
    ---
    Context:
    {context}
    """
)

# PromptTemplate → LLM 순으로 체인 연결
chain = prompt_template | llm

responses = []

# ✅ RAG 방식으로 수정: 질문마다 벡터 검색 → context 만들기 → LLM 호출
for q in prompts[:10]:   # 일단 10개만 테스트
    # 1) 벡터 스토어에서 관련 문장 검색
    retrieved = vector_store.similarity_search(q, k=5)

    # 2) 검색된 문서들의 내용을 하나의 문자열로 합치기
    context_text = "\n\n".join(doc.page_content for doc in retrieved)

    # 3) 체인 실행
    response = chain.invoke({"question": q, "context": context_text})
    responses.append(response.content)

# 출력 확인
for i, resp in enumerate(responses[:10]):
    print(f"[Q{i+1}] {prompts[i]}")
    print(resp)
    print("-" * 50)

[Q1] QUESTION1) 재학 중인 학생이 휴학을 하려면 학기 개시일로부터 며칠 이내에 휴학을 신청하야하나요?
(A) 30일
(B) 45일 
(C) 60일
(D) 90일
The correct answer is **(D) 90일**.  

**Contextual Evidence**:  
According to **제26조(휴학) ⑥** in the provided 학칙 (school regulations):  
> *"재학 중인 자가 휴학을 하고자 하는 경우 학기개시일로부터 90일 이내에 휴학을 신청하여야 한다."*  

This explicitly states that enrolled students must apply for a leave of absence within **90 days** from the start of the semester.  

Thus, the answer is **(D) 90일**.
--------------------------------------------------
[Q2] QUESTION2) '재입학은 a회에 한하여 할 수 있다. 다만 제 28조제4호에 의하여 제적된 자는 제적된 날부터 b년이 경과한 후 재입학 할 수 있다.' a와 b가 상수일 때 a+b의 값을 구하면?
(A) 2
(B) 3
(C) 4
(D) A,B,C 중 답 없음
The context provided includes the relevant information in **제31조(재입학)**:

- **② 재입학은 1회에 한하여 할 수 있다. 다만, 제28조제4호에 의하여 제적된 자는 제적된 날부터 1년이 경과한 후 재입학할 수 있다.**  
  (Translation: "Re-admission is permitted only **once** (a=1). However, those expelled under Article 28(4) may reapply only after **1 year** (b=1) from the date of expulsion.

In [136]:
# funcion to extract an answer from response

import re

def extract_answer(response):
    """
    extracts the answer from the response using a regular expression.
    expected format: "[ANSWER]: (A) convolutional networks"

    if there are any answers formatted like the format, it returns None.
    """
    pattern = r"\[ANSWER\]:\s*\((A|B|C|D|E)\)"
    match = re.search(pattern, response)

    if match:
        return match.group(1) # Extract the letter inside parentheses (e.g., A)
    else:
        return extract_again(response)

def extract_again(response):
    pattern = r"\b[A-J]\b(?!.*\b[A-J]\b)"
    match = re.search(pattern, response)
    if match:
        return match.group(0)
    else:
        return None

In [137]:
# print accuracy

cnt = 0

for answer, response in zip(answers[:10], responses[:10]):
    print("-"*10)
    generated_answer = extract_answer(response)
    # print(response)
    # check
    if generated_answer:
        print(f"generated answer: {generated_answer}, answer: {answer}")
    else:
        print("extraction fail")


    if generated_answer == None:
        continue
    if generated_answer in answer:
        cnt += 1

print()
print(f"acc: {(cnt/10)*100}%")

----------
generated answer: D, answer: (D)
----------
generated answer: A, answer: (A)
----------
generated answer: C, answer: (C)
----------
generated answer: A, answer: (D)
----------
generated answer: C, answer: (C)
----------
generated answer: A, answer: (B)
----------
generated answer: D, answer: (D)
----------
extraction fail
----------
generated answer: C, answer: (C)
----------
generated answer: B, answer: (B)

acc: 70.0%


밑은 틀린문제 보기위한 예시


In [138]:
import pandas as pd
import re

# testset 로드
df_test = pd.read_csv("/content/drive/MyDrive/NLP/testset.csv")

questions = df_test["prompts"].tolist()
golds = df_test["answers"].tolist()

predictions = []

print("====== RAG Evaluation (Q1 ~ Q25) ======\n")

# 25번까지만 평가
for i in range(25):
    q = questions[i]
    gold = golds[i]

    # RAG 검색
    retrieved_docs = vector_store.similarity_search(q, k=5)
    context = "\n".join([d.page_content for d in retrieved_docs])

    # LLM 호출
    answer = chain.invoke({"question": q, "context": context})
    pred_raw = answer.content.strip()

    # 답변에서 (A), (B), (C), (D) 패턴 추출
    match = re.search(r"\(([A-D])\)", pred_raw)
    if match:
        pred = match.group(1)
    else:
        # 선택지 형식이 아니면 그대로 사용
        pred = pred_raw

    predictions.append(pred)

    print(f"Q{i+1}: {q}")
    print(f"Pred: {pred}")
    print(f"Gold: {gold}")
    print("-" * 40)

# 정확도 계산
correct = sum([1 for p, g in zip(predictions, golds[:25]) if str(p).strip() == str(g).strip()])
accuracy = correct / 25 * 100

print(f"\n🎯 Total Accuracy (Q1~25): {accuracy:.2f}%")


====== RAG Evaluation (Q1 ~ Q25) ======

Q1: QUESTION1) 재학 중인 학생이 휴학을 하려면 학기 개시일로부터 며칠 이내에 휴학을 신청하야하나요?
(A) 30일
(B) 45일 
(C) 60일
(D) 90일
Pred: D
Gold: (D)
----------------------------------------
Q2: QUESTION2) '재입학은 a회에 한하여 할 수 있다. 다만 제 28조제4호에 의하여 제적된 자는 제적된 날부터 b년이 경과한 후 재입학 할 수 있다.' a와 b가 상수일 때 a+b의 값을 구하면?
(A) 2
(B) 3
(C) 4
(D) A,B,C 중 답 없음
Pred: A
Gold: (A)
----------------------------------------
Q3: QUESTION3) 학생이 소속 학과 또는 전공 이외의 전공 교과목을 총장이 정하는 바에 따라 몇학점 이상 취득한 때에 부전공을 이수한것으로 인정받을 수 있는가?
(A) 15학점
(B) 18학점
(C) 21학점
(D) 25학점
Pred: C
Gold: (C)
----------------------------------------
Q4: QUESTION4) 다음 보기의 학생들 중 제적을 당하지 않는 사람을 고르면?
(A) 팜 : 징계에 의해 퇴학처분을 받았음 
(B) 엘모 : 성적이 평점평균 1.2 로 학사경고를 연속 3회 받았음
(C) 라마 : 수업료 기타 납입금을 소정 기일 내 납입하지 못하였음
(D) 로라 : 휴학기간 경과 후 3주가 지났으나 갑자기 큰 사고가 났다는 정당한 이유 때문에 복학하지 못하였음
Pred: D
Gold: (D)
----------------------------------------
Q5: QUESTION5) 2019학년도 휴먼기계바이오공학부의 입학 정원은 몇 명인가? 
(A) 90명 
(B) 100명 
(C) 110명 
(D) 120명
Pred: C
Gold: (C)
----------------------

version1-chunk size 900, 50

Q2: QUESTION2) '재입학은 a회에 한하여 할 수 있다. 다만 제 28조제4호에 의하여 제적된 자는 제적된 날부터 b년이 경과한 후 재입학 할 수 있다.' a와 b가 상수일 때 a+b의 값을 구하면?
(A) 2
(B) 3
(C) 4
(D) A,B,C 중 답 없음
Pred: B
Gold: (A)

Q6: QUESTION6) 1980학년도 이전 입학생에 대하여 적용하는 등급에 따른 성적점으로 잘못 연결된 것은 무엇인가?
(A) 등급: A+, 성적점: 4
(B) 등급: A-: 성적점: 3.5
(C) 등급: B+, 성적점: 3
(D) 등급: C, 성적점: 2
Pred: A
Gold: (B)

Q8: QUESTION8) 복수전공 신청 자격에 해당하지 않는 것은?
(A) 1학년을 마친 학생
(B) 평균 평점이 2.50 이상인 학생
(C) 졸업 직전 학기에 있는 학생
(D) 재학생 신분인 경우
Pred: The information is not present in the context.  

**Explanation**:  
The provided context includes general academic regulations (e.g., grading, majors, graduation requirements) but does **not** specify the eligibility criteria for applying for a **복수전공 (double major)**. None of the listed options (A, B, C, or D) are directly addressed in the text regarding application qualifications. Therefore, the correct answer cannot be determined from the given context.  

If the question is part of a test or internal policy not covered here, the answer would require additional information.
Gold: (C)

Q14: QUESTION14) 전공과목은 어떻게 구분되는가?
(A) 전공기초과목과 전공과목
(B) 총장이 따로 정한다
(C) 필수과목과 선택과목
(D) 구분하지 않는다
Pred: A
Gold: (C)

Q23: QUESTION23) 다음중 옳게 짝지어진 학위를 고르세요.
(A) 북한 학과 - 이학사
(B) 기업가정신 - 벤처학사
(C) 미술학사 - 한국음악
(D) 문학사 - 소비자학
Pred: B
Gold: (D)

Q24: QUESTION24) 다음 중 이화여자대학교 학칙에서 학점과 관련해 옳은 것을 고르세요
(A) 교과과정이수 단위는 학기로 한다.
(B) 한 학기동안 10시간 이상 수업을 들으면 1학점을 준다.
(C) 학점취득특별시험의 방법은 총장과 위원회가 함께 정한다.
(D) 제35조는 1999년 2월에 개정되었다.
Pred: C
Gold: (D)

Q25: QUESTION25) 2019학년도 입학 정원에 대한 내용으로 옳은 것을 고르시오.
(A) 휴먼기계바이오공학부 입학 정원과 무용과의 입학 정원의 합은 149명이다.
(B) 건반악기과의 입학정원은 164명이다.
(C) 수학교육과와 국어 교육과의 입학 정원은 각각 37명과 27명이다.
(D) 음악 대학의 학생수는 자연과학대학의 학생수보다 많다.
Pred: A
Gold: (B)